# INLA: Create models and assess predictions
## Models per quartile

In [1]:
#library(rstanarm)
library(INLA)
library(brinla)
library(data.table)
library(ggplot2)
options(repr.plot.width=3, repr.plot.height=3)

Loading required package: sp
Loading required package: Matrix
This is INLA_17.05.31 built 2017-05-31 13:53:28 UTC.
See www.r-inla.org/contact-us for how to get help.


In [2]:
# read data
df = fread('../data/le_cov_sel.csv')
nrow(df)

[1] 12000

In [3]:
print(names(df))

 [1] "le"                   "z_relative_mob"       "z_gini"              
 [4] "county"               "gender"               "income_q"            
 [7] "county_name"          "stateabbrv"           "statename"           
[10] "log_population"       "log_income"           "z_segregation_income"
[13] "log_unemployment"     "z_uninsured"          "z_medicare_expenses" 
[16] "log_crime_rate"       "log_pct_black"        "log_pct_hispanic"    
[19] "z_obesity"            "z_smoking"            "z_exercise"          


In [4]:
df[, state := .GRP, by = statename]
df[, income_qr := .GRP, by = income_q]

In [5]:
table(df[, .(income_qr, income_q)]) # ok, right!

         income_q
income_qr   Q1   Q2   Q3   Q4
        1 3000    0    0    0
        2    0 3000    0    0
        3    0    0 3000    0
        4    0    0    0 3000

# INLA Models (using PC prior)

In [6]:
# create auxiliary variables
df[, state_mob := state]
df[, state_gini := state]
df[, cty := county]
df[, cty_mob := county]
df[, cty_gini := county]
df[, q_mob := income_qr]
df[, q_gini := income_qr]
df[, q_exercise := income_qr]

In [7]:
female = df[gender=='F']
male = df[gender=='M']

# Baseline model

### Male

In [8]:
# same prior per quartile
lmod <- lm(le ~ z_relative_mob  + z_gini + log_population + log_income, male)

# pc prior
sdres <- sd(residuals(lmod))
pcprior <- list(prec = list(prior="pc.prec", param = c(3*sdres,0.01)))

In [9]:
for (i in 1:4) {
    formula = le ~ z_relative_mob  + z_gini + log_population + log_income + 
       f(state, model = "iid", hyper = pcprior)
    model = inla(formula, family = "gaussian", data = male[income_qr==i],
#           control.predictor=list(compute = TRUE),
          control.compute = list(config = TRUE, dic = TRUE,
                                 waic = TRUE, cpo = TRUE), 
#           control.inla = list(strategy ="gaussian"),
          verbose = TRUE)
    
    model_name = paste0('m1_', i)
    assign(model_name, model)
    
    }

In [10]:
m1_1$summary.fixed

,mean,sd,0.025quant,0.5quant,0.975quant,mode,kld
(Intercept),36.4221315,0.12504169,36.1765220,36.4217646,36.6693699,36.4210376,3.635549e-12
z_relative_mob,-0.4865733,0.04037977,-0.5658491,-0.4865794,-0.4073379,-0.4865880,4.255616e-15
z_gini,0.3189737,0.03733470,0.2456574,0.3189743,0.3922186,0.3189787,2.240152e-14
log_population,-0.2542583,0.03939773,-0.3315966,-0.2542677,-0.1769397,-0.2542833,2.682248e-14
log_income,1.1578332,0.18952971,0.7856583,1.1578308,1.5296751,1.1578421,2.163484e-14


In [11]:
bri.hyperpar.summary(m1_1)

,mean,sd,q0.025,q0.5,q0.975,mode
SD for the Gaussian observations,1.081663,0.01997705,1.0431700,1.0813494,1.121600,1.0806705
SD for state,0.815244,0.09513868,0.6473607,0.8082699,1.020416,0.7934126


In [12]:
# simulate values
source('functions/simulation_per_quartile_INLA.R')

In [13]:
nrep = 2 # 4 quartiles for 2 contrast values
relative_mob_pred_data = data.table(
    z_relative_mob = c(0.0, 1.0),
    z_gini = rep(0, nrep),
    log_population = rep(0, nrep), 
    log_income = rep(0, nrep))

In [14]:
sim_male_m1 = data.table()

for (i in 1:4) {

    model_name = paste0('m1_', i)
    s = simulate_predictions_per_quartile(get(model_name), 
                                          relative_mob_pred_data, 
                                          'z_relative_mob', 
                                          nsim = 2000)
    d = s[, .(q = i, fd = diff(value)), by = variable][, .(q, fd)]
    
    sim_male_m1 = rbind(sim_male_m1, d)
    }

In [15]:
fwrite(sim_male_m1, file = '../data/sim_male_m1.csv')

### Female

In [16]:
lmod <- lm(le ~ z_relative_mob  + z_gini + log_population + log_income, female)

# pc prior
sdres <- sd(residuals(lmod))
pcprior <- list(prec = list(prior="pc.prec", param = c(3*sdres,0.01)))

In [17]:
for (i in 1:4) {
    formula = le ~ z_relative_mob  + z_gini + log_population + log_income + 
       f(state, model = "iid", hyper = pcprior)
    model = inla(formula, family = "gaussian", data = female[income_qr==i],
#           control.predictor=list(compute = TRUE),
          control.compute = list(config = TRUE, dic = TRUE,
                                 waic = TRUE, cpo = TRUE), 
#           control.inla = list(strategy ="gaussian"),
          verbose = TRUE)
    
    model_name = paste0('f1_', i)
    assign(model_name, model)
    
    }

In [18]:
f1_1$summary.fixed

,mean,sd,0.025quant,0.5quant,0.975quant,mode,kld
(Intercept),41.9454231,0.10679731,41.7349056,41.9453583,42.1559867,41.9452361,1.495135e-11
z_relative_mob,-0.3462274,0.03967025,-0.4241216,-0.3462298,-0.2683919,-0.3462315,2.425063e-14
z_gini,0.2673734,0.03730382,0.1941223,0.2673725,0.3405618,0.2673737,1.745226e-14
log_population,-0.2453676,0.03932485,-0.3225761,-0.2453724,-0.1682040,-0.2453788,2.355674e-14
log_income,0.6489393,0.18938893,0.2770562,0.6489317,1.0205183,0.6489327,2.205392e-14


In [19]:
bri.hyperpar.summary(f1_1)

,mean,sd,q0.025,q0.5,q0.975,mode
SD for the Gaussian observations,1.0845787,0.02001332,1.0459740,1.0842817,1.1245466,1.0836591
SD for state,0.6833374,0.08319916,0.5361704,0.6774347,0.8624172,0.6656124


In [20]:
sim_female_f1 = data.table()

for (i in 1:4) {

    model_name = paste0('f1_', i)
    s = simulate_predictions_per_quartile(get(model_name), 
                                          relative_mob_pred_data, 
                                          'z_relative_mob', 
                                          nsim = 2000)
    d = s[, .(q = i, fd = diff(value)), by = variable][, .(q, fd)]
    
    sim_female_f1 = rbind(sim_female_f1, d)
    }

In [21]:
fwrite(sim_female_f1, file = '../data/sim_female_f1.csv')

# Adjusting for contextual variables

### Male

In [22]:
# -Demographic: % Black, Hispanic (since Chetty uses this to race adjust)
# -Social: Crime rate, segregation
# -Economic: Gini, possibly unemployment
# -Health Access: uninsured, medicare expenses

In [23]:
lmod <- lm(le ~ z_relative_mob  + z_gini + log_population + log_income + 
           log_crime_rate + z_segregation_income +  log_pct_black + log_pct_hispanic + 
           log_unemployment +  z_uninsured + z_medicare_expenses, male)

# pc prior
sdres <- sd(residuals(lmod))
pcprior <- list(prec = list(prior="pc.prec", param = c(3*sdres,0.01)))

In [24]:
for (i in 1:4) {
    formula = le ~ z_relative_mob  + z_gini + log_population + log_income + 
        log_crime_rate + z_segregation_income +  log_pct_black + log_pct_hispanic + 
        log_unemployment +  z_uninsured + z_medicare_expenses +
        f(state, model = "iid", hyper = pcprior)
    model = inla(formula, family = "gaussian", data = male[income_qr==i],
#           control.predictor=list(compute = TRUE),
          control.compute = list(config = TRUE, dic = TRUE,
                                 waic = TRUE, cpo = TRUE), 
#           control.inla = list(strategy ="gaussian"),
          verbose = TRUE)
    
    model_name = paste0('m2_', i)
    assign(model_name, model)
    
    }

In [25]:
m2_1$summary.fixed

,mean,sd,0.025quant,0.5quant,0.975quant,mode,kld
(Intercept),36.40819077,0.12772163,36.158661841,36.40731747,36.662396442,36.40558537,6.969169e-12
z_relative_mob,-0.39106884,0.04700952,-0.483389524,-0.39106633,-0.298847978,-0.39105729,2.825928e-14
z_gini,0.22616890,0.04252949,0.142669790,0.22616325,0.309621572,0.22615553,2.205859e-14
log_population,-0.07253441,0.05576714,-0.181985427,-0.07255457,0.036926413,-0.07259039,1.213201e-14
log_income,1.66689594,0.22158507,1.231764167,1.66689714,2.101616480,1.66691838,1.808919e-14
log_crime_rate,-0.21794841,0.06325344,-0.342136517,-0.21795644,-0.093831360,-0.21796719,2.167863e-14
z_segregation_income,-0.02413642,0.05258635,-0.127426418,-0.02412762,0.079008889,-0.02410549,1.509472e-14
log_pct_black,-0.06721065,0.03084778,-0.127766327,-0.06721757,-0.006673018,-0.06722885,1.914130e-14
log_pct_hispanic,-0.08308279,0.04337501,-0.168291965,-0.08307149,0.001984611,-0.08304504,2.143749e-14
log_unemployment,0.28563159,0.14912787,-0.007285394,0.28565683,0.578136625,0.28572022,1.872077e-14


In [26]:
bri.hyperpar.summary(m2_1)

,mean,sd,q0.025,q0.5,q0.975,mode
SD for the Gaussian observations,1.0409372,0.01930077,1.003781,1.0406196,1.079555,1.0399182
SD for state,0.8279269,0.10227067,0.648109,0.8202341,1.049101,0.8040011


In [27]:
nrep =  2 # 2 contrast values
relative_mob_pred_data = data.table(
    z_relative_mob       = c(0.0, 1.0),
    z_gini               = rep(0, nrep),
    log_population       = rep(0, nrep), 
    log_income           = rep(0, nrep),
    log_crime_rate       = rep(0, nrep),
#     log_poverty          = rep(0, nrep),
#     log_mig_inflow       = rep(0, nrep),
#     log_mig_outflow      = rep(0, nrep),
#     log_foreign          = rep(0, nrep),
    log_pct_black        = rep(0, nrep),
    log_pct_hispanic     = rep(0, nrep),
#     log_house_value      = rep(0, nrep),
#     log_local_gov_exp    = rep(0, nrep),
    log_unemployment     = rep(0, nrep),
    z_segregation_income = rep(0, nrep),
#     z_religion           = rep(0, nrep),
#     z_labor_force        = rep(0, nrep),
#     z_college            = rep(0, nrep),
#     z_middle_class       = rep(0, nrep),
    z_uninsured          = rep(0, nrep), 
    z_medicare_expenses  = rep(0, nrep))

In [28]:
sim_male_m2 = data.table()

for (i in 1:4) {

    model_name = paste0('m2_', i)
    s = simulate_predictions_per_quartile(get(model_name), 
                                          relative_mob_pred_data, 
                                          'z_relative_mob', 
                                          nsim = 2000)
    d = s[, .(q = i, fd = diff(value)), by = variable][, .(q, fd)]
    
    sim_male_m2 = rbind(sim_male_m2, d)
    }

In [29]:
fwrite(sim_male_m2, file = '../data/sim_male_m2.csv')

### Female

In [30]:
lmod <- lm(le ~ z_relative_mob  + z_gini + log_population + log_income + 
       log_crime_rate + z_segregation_income +  log_pct_black + log_pct_hispanic + 
       log_unemployment +  z_uninsured + z_medicare_expenses, female)

# pc prior
sdres <- sd(residuals(lmod))
pcprior <- list(prec = list(prior="pc.prec", param = c(3*sdres,0.01)))

In [31]:
for (i in 1:4) {
    formula = le ~ z_relative_mob  + z_gini + log_population + log_income + 
        log_crime_rate + z_segregation_income +  log_pct_black + log_pct_hispanic + 
        log_unemployment +  z_uninsured + z_medicare_expenses +
        f(state, model = "iid", hyper = pcprior)
    model = inla(formula, family = "gaussian", data = female[income_qr==i],
#           control.predictor=list(compute = TRUE),
          control.compute = list(config = TRUE, dic = TRUE,
                                 waic = TRUE, cpo = TRUE), 
#           control.inla = list(strategy ="gaussian"),
          verbose = TRUE)
    
    model_name = paste0('f2_', i)
    assign(model_name, model)
    
    }

In [32]:
f2_1$summary.fixed

,mean,sd,0.025quant,0.5quant,0.975quant,mode,kld
(Intercept),41.93323527,0.10655401,41.72343681,41.93307091,4.214362e+01,41.93275629,7.886406e-19
z_relative_mob,-0.27786995,0.04701858,-0.37019769,-0.27787106,-1.856219e-01,-0.27786930,2.354032e-14
z_gini,0.26144613,0.04253009,0.17793656,0.26144369,3.448916e-01,0.26144238,2.109893e-14
log_population,0.02852736,0.05517615,-0.07981945,0.02852634,1.367791e-01,0.02852896,1.969394e-14
log_income,0.95109209,0.22045042,0.51823957,0.95107548,1.383634e+00,0.95106082,2.227373e-14
log_crime_rate,-0.31948515,0.06325298,-0.44368501,-0.31948882,-1.953805e-01,-0.31949082,2.167895e-14
z_segregation_income,-0.10303702,0.05251248,-0.20616188,-0.10303505,-1.892891e-05,-0.10302662,1.604151e-14
log_pct_black,-0.07427767,0.03053759,-0.13425814,-0.07427316,-1.437792e-02,-0.07426151,2.278750e-14
log_pct_hispanic,-0.20946950,0.04284523,-0.29360724,-0.20946931,-1.254112e-01,-0.20946527,2.551461e-14
log_unemployment,0.51107487,0.14742028,0.22159594,0.51106970,8.003128e-01,0.51107187,2.554264e-14


In [33]:
bri.hyperpar.summary(f2_1)

,mean,sd,q0.025,q0.5,q0.975,mode
SD for the Gaussian observations,1.0460784,0.01937325,1.0087380,1.0457785,1.0847973,1.0451351
SD for state,0.6769903,0.08466052,0.5278019,0.6707777,0.8597488,0.6580783


In [34]:
sim_female_m2 = data.table()

for (i in 1:4) {

    model_name = paste0('f2_', i)
    s = simulate_predictions_per_quartile(get(model_name), 
                                          relative_mob_pred_data, 
                                          'z_relative_mob', 
                                          nsim = 2000)
    d = s[, .(q = i, fd = diff(value)), by = variable][, .(q, fd)]
    
    sim_female_m2 = rbind(sim_female_m2, d)
    }

In [35]:
fwrite(sim_female_m2, file = '../data/sim_female_m2.csv')

# Create tables

In [40]:
library(texreg)
source('functions/extract_inla.R')

In [41]:
for (i in 1:4) {
    cmodels <- rep(c('Women', 'Men'), 2)
    models <- list(get(paste0('f1_', i)),
                   get(paste0('m1_', i)),
                   get(paste0('f2_', i)),
                   get(paste0('m2_', i)))

    cnames <- list("(Intercept)" = 'Constant',
                   'z_relative_mob' = 'Standardized Income mobility (Rank-Rank Slope)',
                   'z_gini' = 'Standardized Gini',
                   "sd for state" = "SD states",
                   "sd for the Gaussian observations" = "SD observations")

    # screenreg(models)
    t = texreg(models,
                include.dic = TRUE, include.waic = TRUE,
                ci.test = FALSE,
                float.pos = "htp",
                caption = "Life Expectancy (40) Models",
                booktabs = TRUE,
                use.packages = FALSE,
                dcolumn = TRUE,
                caption.above = TRUE,
                scalebox = 0.65,
                label = "inla_models",
                # sideways = TRUE,
                digits = 2,
                custom.model.names = cmodels,
                custom.coef.map = cnames,
                groups = list("Random Effects" = c(4:5)),
                custom.note = "Note: Selected coefficients 
                (mean of marginal posterior distribution). 95\\% credibility intervals.")
    
    assign(paste0('tab_', i), t)
    
}

In [79]:
heading = '\\renewcommand{\\arraystretch}{1.6}\n\\begin{table}[htp]\n\\caption{Life Expectancy (40) Models}\n\\begin{center}\n\\scalebox{0.65}{\n\\begin{tabular}{l D{.}{.}{5.11} D{.}{.}{5.11} D{.}{.}{5.11} D{.}{.}{5.11} }\n\\toprule\n& \\multicolumn{2}{c}{Baseline} & \\multicolumn{2}{c}{Social Indicators} \\\\\n& \\multicolumn{1}{c}{Women} & \\multicolumn{1}{c}{Men} & \\multicolumn{1}{c}{Women} & \\multicolumn{1}{c}{Men} \\\\\n\\midrule\n'

In [80]:
bottom = '\\addlinespace[5pt]\n\\bottomrule\n\\multicolumn{5}{l}{\\scriptsize{Note: Selected coefficients(mean of marginal posterior distribution). 95\\% credibility intervals.}}\n\\end{tabular}\n}\n\\label{inla_models}\n\\end{center}\n\\end{table}\n'

In [89]:
sep = NA
for (i in 1:4) {
  sep[i] = paste0("\\addlinespace[10pt]\n\\multicolumn{5}{l}{\\textbf{Income Quartile ", i, "}} \\\\\n\\addlinespace[10pt]\n")
}

In [90]:
tabs = list(tab_1, tab_2, tab_3, tab_4)

In [91]:
out = list()
for (i in 1:4) {
    out[[i]] = gsub('(.+)(Constant.+)(Random.+)', '\\2', tabs[[i]])
}

In [92]:
cat(heading, sep[[1]], out[[1]], sep[[2]], out[[2]], sep[[3]], out[[3]], sep[[4]], 
    out[[4]], bottom, file = 'tables/inla_models_quartile.tex')

# Prediction per county

In [488]:
library(sdazar)
library(ggridges)
library(forcats)
library(ggthemes)

Loading required package: knitr
Loading required package: miceadds
Loading required package: mice
* miceadds 2.5-9 (2017-06-17 14:42:44)


In [523]:
setorder(male, le)

In [524]:
setorder(female, le)

In [487]:
df[, cn := paste0(county_name, ", ", stateabbrv)]
cn = df[, .(cn, state, county)]
cn = cn[!duplicated(cn)]

In [494]:
vars = c('le', 'county', 'state', 'z_relative_mob', 'z_gini', 'log_population', 'log_income' ,  
        'log_crime_rate', 'z_segregation_income', 'log_pct_black', 'log_pct_hispanic', 
        'log_unemployment', 'z_uninsured', 'z_medicare_expenses')

### Male

In [495]:
min_mob = min(female$z_relative_mob)

In [503]:
bottom_male = head(male[income_qr==1, vars, with=FALSE], 10)
bottom_male_c = copy(bottom_male)
bottom_male_c[, z_relative_mob := min_mob]
tot_bottom_male = rbind(bottom_male, bottom_male_c) 
tot_bottom_male[, ranking := .GRP, by = .(county, state)]
tot_bottom_male[, order := seq_len(.N), by = .(county, state)]
anyDuplicated(tot_bottom_male[,.(state, county, z_relative_mob)])

In [504]:
# simulate values
source('functions/simulation_random_intercept.R')

In [508]:
sim_male_bottom = simulate_predictions(m2_1, nsim=2000, tot_bottom, contrast='z_relative_mob', 
                  random_intercept='state')

sim_male_bottom = merge(sim_male_bottom, cn, by = c('county', 'state'))

setorder(sim_male_bottom, ranking)

head(sim_male_bottom)

In [511]:
sdazar::savepdf('plots/counties_male')
plot = ggplot(sim_male_bottom[ranking<9], aes(y = cn)) + 
  geom_density_ridges(aes(x = pred, fill = paste(order)), 
           alpha = .5, color = "white", from = 33, to = 38) +
  labs(x = "E(40)",
       y = "County") +  theme_few() +
#        title = "Indy vs Unionist vote in Catalan elections",
#        subtitle = "Analysis unit: municipalities (n = 949)",
#        caption = "Marc Belzunces (@marcbeldata) | Source: Idescat") +
  scale_y_discrete(expand = c(0.05, 0)) +
  scale_x_continuous(expand = c(0.000001, 0))  +
  scale_fill_cyclical(values = c("#ff0000", "#0000ff"))
print(plot)
dev.off()

Picking joint bandwidth of 0.0493


pdf 
  2

### Female

In [512]:
min_mob = min(female$z_relative_mob)

In [528]:
head(bottom_female, 3)

le,county,state,z_relative_mob,z_gini,log_population,log_income,log_crime_rate,z_segregation_income,log_pct_black,log_pct_hispanic,log_unemployment,z_uninsured,z_medicare_expenses
37.88754,18133,12,0.1347668,-1.0845401,-0.8031524,-0.16104698,0.8230133,-0.9515619,-0.03726077,-0.89118654,-0.25317740,-0.1622971,0.3946781
38.01845,48273,41,-0.4477979,0.4658374,-0.9356571,-0.33170223,-0.2183776,-0.1877166,0.13927913,3.15503860,0.06280708,1.7337207,2.0011309
38.33007,18071,12,-0.1875013,-0.6893700,-0.6654894,0.01256847,-0.3776445,-0.4850299,-1.74584115,-0.03595746,-0.15761256,0.3274003,-0.8805983


In [527]:
head(bottom_male, 3)

le,county,state,z_relative_mob,z_gini,log_population,log_income,log_crime_rate,z_segregation_income,log_pct_black,log_pct_hispanic,log_unemployment,z_uninsured,z_medicare_expenses
31.51167,48025,41,0.03526187,0.04354253,-0.9103068,-0.6552334,-0.4113431,-0.4929434,1.1726481,2.962039,0.24400568,1.1367667,1.9863269
31.86600,48273,41,-0.44779786,0.46583745,-0.9356571,-0.3317022,-0.2183776,-0.1877166,0.1392791,3.155039,0.06280708,1.7337207,2.0011309
32.36390,51149,44,-0.57220781,-1.95504689,-0.8892682,-0.1012650,-1.5022154,-0.7503726,2.3659887,0.567169,-0.31159711,-0.8624501,0.0683768


In [513]:
bottom_female = head(female[income_qr==1, vars, with=FALSE], 10)

bottom_female_c = copy(bottom_female)

bottom_female_c[, z_relative_mob := min_mob]

tot_bottom_female = rbind(bottom_female, bottom_female_c) 

tot_bottom_female[, ranking := .GRP, by = .(county, state)]
tot_bottom_female[, order := seq_len(.N), by = .(county, state)]

anyDuplicated(tot_bottom_female[,.(state, county, z_relative_mob)])

In [514]:
sim_female_bottom = simulate_predictions(f2_1, nsim=2000, tot_bottom, contrast='z_relative_mob', 
                  random_intercept='state')

sim_female_bottom = merge(sim_female_bottom, cn, by = c('county', 'state'))

setorder(sim_female_bottom, ranking)

head(sim_female_bottom)

county,state,pred,sim,z_relative_mob,le,ranking,order,cn
48025,41,39.92849,1,0.03526187,31.51167,1,1,"Bee, TX"
48025,41,41.08358,1,-3.53846431,31.51167,1,2,"Bee, TX"
48025,41,39.74280,2,0.03526187,31.51167,1,1,"Bee, TX"
48025,41,40.75298,2,-3.53846431,31.51167,1,2,"Bee, TX"
48025,41,40.34960,3,0.03526187,31.51167,1,1,"Bee, TX"
48025,41,41.42009,3,-3.53846431,31.51167,1,2,"Bee, TX"


In [529]:
sdazar::savepdf('plots/counties_female')
plot = ggplot(sim_female_bottom[ranking<9], aes(y = cn)) + 
  geom_density_ridges(aes(x = pred, fill = paste(order)), 
           alpha = .5, color = "white", from = 39, to = 44) +
  labs(x = "E(40)",
       y = "County") +  theme_few() +
#        title = "Indy vs Unionist vote in Catalan elections",
#        subtitle = "Analysis unit: municipalities (n = 949)",
#        caption = "Marc Belzunces (@marcbeldata) | Source: Idescat") +
  scale_y_discrete(expand = c(0.05, 0)) +
  scale_x_continuous(expand = c(0.000001, 0))  +
  scale_fill_cyclical(values = c("#ff0000", "#0000ff"))
print(plot)
dev.off()

Picking joint bandwidth of 0.0484


pdf 
  2